In [2]:
import pandas as pd
import json

In [3]:

# Load dataset
file_path = 'diabetes_binary_5050split_health_indicators_BRFSS2015.csv'
df = pd.read_csv(file_path)

df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [ ]:
#edits

if 'Sex' in df.columns:
    df['Sex'] = df['Sex'].replace({0: 'Female', 1: 'Male'})

if 'Age' in df.columns:
    df['Age'] = df['Age'].replace({
        1: "18-24",
        2: "25-29",
        3: "30-34",
        4: "35-39",
        5: "40-44",
        6: "45-49",
        7: "50-54",
        8: "55-59",
        9: "60-64",
        10: "65-69",
        11: "70-74",
        12: "75-79",
        13: "80+"
    })

#scale 1-8 1 = less than $10,000 5 = less than $35,000 8 = $75,000 or more"}]
if 'Income' in df.columns:
    df['Income'] = df['Income'].replace({
        1: "< $10,000",
        2: "$10,000 - $15,000",
        3: "$15,000 - $20,000",
        4: "$20,000 - $25,000",
        5: "$25,000 - $35,000",
        6: "$35,000 - $50,000",
        7: "$50,000 - $75,000",
        8: "> $75,000"
    })

#scale 1-6 1 = Never attended school or only kindergarten 2 = elementary etc.
if 'Education' in df.columns:
    df['Education'] = df['Education'].replace({
        1: "Never attended school or Kindergarten",
        2: "Elementary School",
        3: "Some high school",
        4: "High school graduate",
        5: "Some college or technical school",
        6: "College graduate"
    })


        
    

In [47]:
demographics = ['Sex', 'Age', 'Education', 'Income']

result = []

for demographic in demographics:
    grouped = (
        df.groupby(demographic)
        .apply(lambda g: pd.Series({
            "Subgroup_total": len(g),
            "Diabetes_binary_count": (g["Diabetes_binary"] == 1).sum(),
            "NonDiabetes_count": (g["Diabetes_binary"] == 0).sum()
        }))
        .reset_index()
    )

    # Total population 
    total_population = df.shape[0]

    # Bar height: each categroy/total population
    grouped["Category_percentage"] = (grouped["Subgroup_total"] / total_population) * 100


#category = income, sex, eduction...
#subgroup = groups within categories, like male/female, age groups 
    
    # Bar colors: diabetes/ each subgroup & no diabetes/each subgroup
    grouped["Diabetes"] = (grouped["Diabetes_binary_count"] / grouped["Subgroup_total"]) * 100
    grouped["NonDiabetes"] = (grouped["NonDiabetes_count"] / grouped["Subgroup_total"]) * 100

    grouped["Demographic"] = demographic
    grouped.rename(columns={demographic: "Category"}, inplace=True)
    result.append(grouped)

'''
#processing
for demographic in demographics:
    grouped = (
        df.groupby(demographic)
        .apply(lambda g: pd.Series({
            "Subgroup_total": len(g),
            "Diabetes_binary_count": (g["Diabetes_binary"] == 1).sum(),
            "NonDiabetes_count": (g["Diabetes_binary"] == 0).sum()
        }))
        .reset_index()
    )

    # Calculate proportions
    total_population = grouped["Subgroup_total"].sum()
    grouped["Subgroup_percentage"] = (grouped["Subgroup_total"] / total_population) * 100
    grouped["Diabetes"] = (grouped["Diabetes_binary_count"] / grouped["Subgroup_total"]) * grouped["Subgroup_percentage"]
    grouped["NonDiabetes"] = (grouped["NonDiabetes_count"] / grouped["Subgroup_total"]) * grouped["Subgroup_percentage"]


    grouped["Demographic"] = demographic
    grouped.rename(columns={demographic: "Category"}, inplace=True)
    result.append(grouped)

'''

'\n#processing\nfor demographic in demographics:\n    grouped = (\n        df.groupby(demographic)\n        .apply(lambda g: pd.Series({\n            "Subgroup_total": len(g),\n            "Diabetes_binary_count": (g["Diabetes_binary"] == 1).sum(),\n            "NonDiabetes_count": (g["Diabetes_binary"] == 0).sum()\n        }))\n        .reset_index()\n    )\n\n    # Calculate proportions\n    total_population = grouped["Subgroup_total"].sum()\n    grouped["Subgroup_percentage"] = (grouped["Subgroup_total"] / total_population) * 100\n    grouped["Diabetes"] = (grouped["Diabetes_binary_count"] / grouped["Subgroup_total"]) * grouped["Subgroup_percentage"]\n    grouped["NonDiabetes"] = (grouped["NonDiabetes_count"] / grouped["Subgroup_total"]) * grouped["Subgroup_percentage"]\n\n\n    grouped["Demographic"] = demographic\n    grouped.rename(columns={demographic: "Category"}, inplace=True)\n    result.append(grouped)\n\n'

In [48]:
# Combine all demographics into one DataFrame
final_df = pd.concat(result)
final_df.head(30)

,Category,Subgroup_total,Diabetes_binary_count,NonDiabetes_count,Category_percentage,Diabetes,NonDiabetes,Demographic
0,Female,38386,18411,19975,54.300345,47.962799,52.037201,Sex
1,Male,32306,16935,15371,45.699655,52.420603,47.579397,Sex
0,18-24,979,78,901,1.384881,7.967314,92.032686,Age
1,25-29,1396,140,1256,1.974764,10.028653,89.971347,Age
2,30-34,2049,314,1735,2.898489,15.324549,84.675451,Age
3,35-39,2793,626,2167,3.950942,22.413176,77.586824,Age
4,40-44,3520,1051,2469,4.979347,29.857955,70.142045,Age
5,45-49,4648,1742,2906,6.575001,37.478485,62.521515,Age
6,50-54,6872,3088,3784,9.721043,44.935972,55.064028,Age
7,55-59,8603,4263,4340,12.169694,49.552482,50.447518,Age


In [49]:
# Save to JSON 
final_df.to_json("preprocessed_demographic_data.json", orient="records")